In [7]:
# import dependenices.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import requests
from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [8]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [9]:
#previww coordinates and structure.
coordinates = list(lat_lngs)
print(coordinates[0:3])

[(53.747361496395, 72.36992867980567), (46.316532516433995, 152.06283262069854), (76.20017240659627, -150.71084011025653)]


In [10]:
# Create a list to hold the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

717

In [11]:
# preview city data 
cities[0:5]

['odesskoye', 'sentyabrskiy', 'barrow', 'bilibino', 'isangel']

In [12]:
# create endpoint url API looping
url = 'http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=' + weather_api_key 

In [13]:
r = requests.get(url + "&q=" + cities[13].replace(" ","+") ).json()

In [14]:
r

{'coord': {'lon': 99.1328, 'lat': 54.8318},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': -1.08,
  'feels_like': -13.68,
  'temp_min': -1.08,
  'temp_max': -1.08,
  'pressure': 1036,
  'humidity': 83,
  'sea_level': 1036,
  'grnd_level': 979},
 'visibility': 10000,
 'wind': {'speed': 6.96, 'deg': 127, 'gust': 11.65},
 'clouds': {'all': 58},
 'dt': 1644732388,
 'sys': {'country': 'RU', 'sunrise': 1644713459, 'sunset': 1644747876},
 'timezone': 28800,
 'id': 1491977,
 'name': 'Shumskiy',
 'cod': 200}

In [17]:
# Create an empty list to hold the weather data.
city_data = []
skip_count = 0
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        # Change the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        skip_count += 1
        continue
# print api summary notes
print('{} cities skipped.'.format(skip_count)) 
print('data for {} cities acquired '.format(len(city_data)))

# this took forever to load

65 cities skipped.
data for 652 cities acquired 


In [18]:
# review dataframe
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Odesskoye,RU,54.2111,72.9651,7.47,94,95,6.31,overcast clouds
1,Barrow,US,71.2906,-156.7887,-29.18,75,0,8.05,clear sky
2,Bilibino,RU,68.0546,166.4372,-25.10,100,100,1.83,overcast clouds
3,Isangel,VU,-19.5500,169.2667,81.84,81,100,4.99,overcast clouds
4,Bengkulu,ID,-3.8004,102.2655,84.16,69,97,7.63,overcast clouds


In [19]:
# Export the City_Data_DF into a CSV file
city_data_df.to_csv('WeatherPy_Database.csv',index_label='City_ID')